In [ ]:
### Emma Beyer & Desa Bolger
### emma.beyer@duke.edu & desa.bolger@duke.edu

#The goal of this document is to explore the relation of wrecks in NC and presence of military bases

#Read in packages
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import Polygon
import matplotlib.pyplot as plt

In [ ]:
#Read Wrecks in NC EEZ into a Pandas dataframe
NC_Wrecks = gpd.read_file('../Data/Processed/ONLYncWRECKS.shp')
NC_Wrecks.head()

In [ ]:
#Read Miliary Bases into a Pandas dataframe
US_Bases = gpd.read_file('../Data/Raw/tl_2024_us_mil.shp')
US_Bases.head()

In [ ]:
#Check coordinate systems
NC_Wrecks.crs == US_Bases.crs

In [ ]:
#Change US_Bases to fit Wreck data
print("Current CRS:", US_Bases.crs)
US_Bases = US_Bases.to_crs("EPSG:4326")
print("Current CRS:", US_Bases.crs) 

#Recheck coordinate systems
print(NC_Wrecks.crs == US_Bases.crs)

In [ ]:
#Read in NC counties into a Pandas dataframe
NC_Waters = gpd.read_file('../Data/Processed/NCeezwithin200.shp')

#Subset US Bases to only include the ones that touch NC Federal Waters
NC_CoastalBases = gpd.sjoin(US_Bases, NC_Waters, how='inner', predicate='intersects')

#Check new subset
print(NC_CoastalBases.plot())

#Save new NC Coastal Bases shapefile
NC_CoastalBases.to_file("../Data/Processed/NC_CoastalBases.shp")

In [ ]:
#Create a 20 nm buffer of the Miliary Bases

#Reproject NC_CoastalBases GeoDataFrame to a projected CRS (UTM zone 17N (EPSG:32617) for North Carolina)
NC_CoastalBases_projected = NC_CoastalBases.to_crs(epsg=32617)

#Create a 20 nm (92600 m) buffer around the military bases in the projected CRS
NC_Bases_buffer_projected = NC_CoastalBases_projected.buffer(37040)  

#Convert the buffer back to the original CRS 
NC_Bases_buffer_gdf = gpd.GeoDataFrame(geometry=NC_Bases_buffer_projected, crs=NC_CoastalBases_projected.crs)
NC_Bases_buffer_gdf = NC_Bases_buffer_gdf.to_crs(NC_CoastalBases.crs)  

#Plot the results
NC_Bases_buffer_gdf.plot()

In [ ]:
#Subset NC Wrecks to only include the ones that are within the 50 nm buffer
WrecksWithinBuffer = gpd.sjoin(NC_Wrecks, NC_Bases_buffer_gdf, how='inner', predicate='within')

#Check counts of wrecks
print(f"There were {len(WrecksWithinBuffer)} wrecks within 20 nm of North Carolina Mililary Bases.")
print(f"There were {len(NC_Wrecks)} wrecks within the entire North Carolina Federal Waters.")

#Note that the new buffered points included duplicates and needs to be updates

In [ ]:
#Remove the duplicated points
Unique_WrecksWithinBuffer = WrecksWithinBuffer.drop_duplicates(subset='geometry')

#Check the counts of wrecks again
print(f"There were {len(Unique_WrecksWithinBuffer)} wrecks within 20 nm of North Carolina Military Bases.")
print(f"There were {len(NC_Wrecks)} wrecks within the entire North Carolina Federal Waters.")
print(f"That means that {(len(Unique_WrecksWithinBuffer)) / (len(NC_Wrecks)) * 100}% of wrecks happened within 20 nm of a military base.")

In [ ]:
#Read in NC Federal Waters into a Pandas dataframe
NC_EEZ = gpd.read_file('../Data/Processed/AreaOfInterest.shp')

#Plot of Federal Waters
ax = NC_EEZ.plot(color='lightblue', edgecolor='black', figsize=(10, 10))

#Plot of All wrecks within federal waters
NC_Wrecks.plot(ax=ax, color='lightgrey', edgecolor='black', markersize=20)

#Plot of Wrecks within 20 nm of military bases
Unique_WrecksWithinBuffer.plot(ax=ax, color='red', edgecolor='black', markersize=20)

#Plot of Military Bases
NC_CoastalBases.plot(ax=ax, color='yellow', edgecolor='black')
plt.title('Wrecks within 20 nm of NC Mililary Bases')

#Create custom legend
handles = [
    plt.Line2D([0], [0], marker='o', color='lightblue', label='Study Area', markersize=10),
    plt.Line2D([0], [0], marker='o', color='lightgrey', label='Wrecks within Study Area', markersize=10),
    plt.Line2D([0], [0], marker='o', color='red', label='Wrecks within 20 nm of Military Bases', markersize=10),
    plt.Line2D([0], [0], marker='o', color='yellow', label='Military Bases', markersize=10)
]

#Adding the legend
ax.legend(handles=handles, loc='upper left', title='Legend')

#Save plot to png
plt.savefig('../Products/NC_Military_Wrecks.png', dpi=300)